### 1. Libraries

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import FirefoxOptions

In [66]:
opts = FirefoxOptions()
opts.add_argument('--headless')
browser = webdriver.Firefox(options=opts)

In [3]:
import pandas as pd

### 2. Reading vacancies' urls

In [92]:
df = pd.read_csv("aijobs3.csv", header=None)

In [93]:
df.head()

,0,1,2,3,4,5,6,7
0,https://ai-jobs.net/job/31060-director-enginee...,"View full details of `Director, Engineering Da...","Austin, TX",Full Time,Senior-level,USD 230K - 345K,NaN,Glassdoor
1,https://ai-jobs.net/job/31141-data-operations-...,View full details of `Data Operations Analyst ...,Hyderabad - India,Full Time,Mid-level,NaN,NaN,Dun & Bradstreet
2,https://ai-jobs.net/job/31132-data-engineer-re...,View full details of `Data Engineer (Remote)` ...,Brazil - Remote,Full Time,NaN,USD 42K - 78K,NaN,Blue Orange Digital
3,https://ai-jobs.net/job/31079-data-analyst-ris...,"View full details of `Data Analyst, Risk Opera...","Bengaluru, Karnataka",Full Time,Mid-level,USD 51K - 90K *,NaN,Greenlight
4,https://ai-jobs.net/job/31053-pm-data-strategy...,"View full details of `PM, Data Strategy Manage...","North Reading, Massachusetts, USA",Full Time,Senior-level,NaN,NaN,Amazon.com


In [19]:
df.iloc[:, [0]]

,0
0,https://ai-jobs.net/job/32218-data-science-con...
1,https://ai-jobs.net/job/32029-python-php-senio...
2,https://ai-jobs.net/job/30723-data-scientist-r...
3,https://ai-jobs.net/job/30166-postdoctoral-res...
4,https://ai-jobs.net/job/30091-data-science-and...
...,...
995,https://ai-jobs.net/job/32175-data-scientist/
996,https://ai-jobs.net/job/32212-senior-data-arch...
997,https://ai-jobs.net/job/32166-data-scientist-b...
998,https://ai-jobs.net/job/32156-data-analyst-gro...


In [94]:
urls = list(df[df.columns[0]])

In [95]:
urls

['https://ai-jobs.net/job/31060-director-engineering-data-architecture/',
 'https://ai-jobs.net/job/31141-data-operations-analyst-ii-r-13164/',
 'https://ai-jobs.net/job/31132-data-engineer-remote/',
 'https://ai-jobs.net/job/31079-data-analyst-risk-operations/',
 'https://ai-jobs.net/job/31053-pm-data-strategy-manager-robotics-supply-chain/',
 'https://ai-jobs.net/job/31080-business-data-analyst-intern-finance/',
 'https://ai-jobs.net/job/31109-machine-learning-engineer-big-data/',
 'https://ai-jobs.net/job/31066-junior-technical-solutions-engineer-data/',
 'https://ai-jobs.net/job/31095-senior-data-scientist/',
 'https://ai-jobs.net/job/31063-artificial-intelligence-researcher-internship/',
 'https://ai-jobs.net/job/31068-senior-business-data-analyst-all-genders/',
 'https://ai-jobs.net/job/31073-sr-data-analyst/',
 'https://ai-jobs.net/job/31082-lead-data-analyst/',
 'https://ai-jobs.net/job/31129-lead-data-engineer-peacock/',
 'https://ai-jobs.net/job/31117-machine-learning-enginee

In [96]:
len(urls)

1754

In [132]:
urls[1753]

'https://ai-jobs.net/job/29195-quality-assurance-engineer-intern-big-data/'

### 3. Scraping vacancies for descriptions and additional info

In [27]:
import time
from tqdm.auto import tqdm
import json

In [45]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [97]:
descriptions3 = []

for url in tqdm(urls):
    URL = url
    browser.get(URL)
    time.sleep(.5)
    
    job_dict = {}
    
    #browser.find_element('xpath', 'html').text
    text = browser.find_element(By.XPATH, "//script[@type='application/ld+json']")
    try:
        data = json.loads(text.get_attribute('innerHTML'))
        cont = 1
    except Exception as e:
        cont = 0
    
    if cont:
        try:
            job_dict['minSalary'] = data['baseSalary']['value']['minValue']
        except KeyError:
            job_dict['minSalary'] = None

        try:   
            job_dict['maxSalary'] = data['baseSalary']['value']['maxValue']
        except KeyError:
            job_dict['maxSalary'] = None

        try:
            job_dict['currency'] = data['baseSalary']['currency']
        except KeyError:
            job_dict['currency'] = None

        try:
            job_dict['salaryPeriod'] = data['baseSalary']['value']['unitText']
        except KeyError:
            job_dict['salaryPeriod'] = None

        try:
            job_dict['position'] = data['title']
        except KeyError:
            job_dict['position'] = None

        try:
            job_dict['datePosted'] = data['datePosted']
        except KeyError:
            job_dict['datePosted'] = None

        try:
            job_dict['validThrough'] = data['validThrough']
        except KeyError:
            job_dict['validThrough'] = None
        
        try:
            job_dict['jobDescription'] = data['description']
        except KeyError:
            job_dict['jobDescription'] = None
        

    else:
        job_dict['minSalary'] = None
        job_dict['maxSalary'] = None
        job_dict['currency'] = None
        job_dict['salaryPeriod'] = None
        job_dict['position'] = None
        job_dict['datePosted'] = None
        job_dict['validThrough'] = None
        job_dict['jobDescription'] = None
    
    skills = []
    try:
        for el in browser.find_elements(By.CSS_SELECTOR, "[title^='Find more jobs related to ']"):
            skills.append(el.text)
    except NoSuchElementException:
        skills = []
    
    job_dict['skills'] = skills
    
    descriptions3.append(job_dict)

  0%|          | 0/1754 [00:00<?, ?it/s]

In [98]:
des = pd.DataFrame(descriptions3)

In [99]:
des

,minSalary,maxSalary,currency,salaryPeriod,position,datePosted,validThrough,jobDescription,skills
0,230400.0,345600.0,USD,YEAR,"Director, Engineering Data Architecture",2022-10-31 16:35:17,2022-12-15 00:00:00,Why Glassdoor? Our mission is to help people e...,"[Agile, Airflow, AWS, Big Data, Computer Scien..."
1,NaN,NaN,None,None,Data Operations Analyst II (R-13164),2022-10-31 15:58:11,2022-12-15 00:00:00,Why We Work at Dun &amp;amp; BradstreetDun &am...,"[Agile, AWS, Computer Science, Data analysis, ..."
2,42000.0,78000.0,USD,YEAR,Data Engineer (Remote),2022-10-31 15:56:06,2022-12-15 00:00:00,"This is a full-time, fully remote role for Lat...","[Airflow, AWS, Big Data, Bitbucket, Computer S..."
3,51519.0,90000.0,USD,YEAR,"Data Analyst, Risk Operations",2022-10-31 15:44:29,2022-12-15 00:00:00,Greenlight is a fintech company that makes a d...,"[Data visualization, Data Warehousing, ELT, ET..."
4,NaN,NaN,None,None,"PM, Data Strategy Manager, Robotics Supply Chain",2022-10-31 15:44:04,2022-12-15 00:00:00,Job summaryAre you inspired by invention? Is p...,"[Data strategy, Engineering, Finance, Machine ..."
...,...,...,...,...,...,...,...,...,...
1749,NaN,NaN,None,None,Fleet Data Manager SME,2022-10-05 01:55:47,2022-11-19 00:00:00,"SOLUTE, Inc., a Sigma Defense Systems company,...","[Data management, Security]"
1750,NaN,NaN,None,None,[Job- 8581] Data Engineer Pleno,2022-10-05 01:41:53,2022-11-19 00:00:00,"CI&amp;amp;T é uma digital specialist, parceir...","[Airflow, Dataproc, ELT, ETL, GCP, Pipelines, ..."
1751,NaN,NaN,None,None,Research Scientist in ML for Climate Modeling,2022-10-05 01:20:29,2022-11-19 00:00:00,Remote hybrid: Persons in these roles are expe...,"[Machine Learning, Mathematics, NLP, Python, R..."
1752,NaN,NaN,None,None,Software Engineer - Market Data Team,2022-10-05 00:14:03,2022-11-19 00:00:00,We are seeking a Market Data Engineer to help ...,"[Big Data, Git, OOP, Research, SQL, STEM]"


### 4. Saving to CSVs

In [82]:
import csv

In [100]:
with open('descriptions3.csv','w') as f:
    w = csv.DictWriter(f, job_dict.keys())
    w.writerows(descriptions3)
    f.close()

### 5. Merging descriptions

In [101]:
all_filenames = ['descriptions1.csv', 'descriptions2.csv', 'descriptions3.csv']

In [118]:
descr1 = pd.read_csv("descriptions1.csv", header=None)

In [119]:
descr2 = pd.read_csv("descriptions2.csv", header=None)

In [120]:
descr3 = pd.read_csv("descriptions3.csv", header=None)

In [123]:
concat = pd.concat([descr1, descr2, descr3], ignore_index=True)

In [126]:
concat

,minSalary,maxSalary,currency,salaryPeriod,position,datePosted,datePosted,jobDescription,skills
0,9000.0,11000.0,USD,YEAR,Data Science Content Intern (Remote),2022-11-09 22:55:46,2022-12-24 00:00:00,Description NannyML - creators of an Open Sour...,"['Bayesian', 'Content creation', 'GitHub', 'Ma..."
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['APIs', 'GCP', 'Google Cloud', 'Machine Learn..."
2,55000.0,120000.0,USD,YEAR,"Data Scientist - Research, Development &amp; C...",2022-10-26 20:46:21,2022-12-10 00:00:00,"Company Description: HFR, Inc. (HFR) was fo...","['Computer Science', 'Data Analytics', 'Data v..."
3,NaN,NaN,NaN,NaN,Postdoctoral Research Associate - Boundary Pla...,2022-10-21 20:53:22,2022-12-05 00:00:00,Overview: Oak Ridge National Laboratory is th...,"['CAD', 'Engineering', 'Matlab', 'Physics', 'P..."
4,150000.0,200000.0,USD,YEAR,Data Science and Machine Learning Manager,2022-10-21 02:15:41,2022-12-05 00:00:00,USA Remote Position POSITION SUMMARY: The Dat...,"['Agile', 'Airflow', 'APIs', 'AWS', 'CI/CD', '..."
...,...,...,...,...,...,...,...,...,...
3749,NaN,NaN,NaN,NaN,Fleet Data Manager SME,2022-10-05 01:55:47,2022-11-19 00:00:00,"SOLUTE, Inc., a Sigma Defense Systems company,...","['Data management', 'Security']"
3750,NaN,NaN,NaN,NaN,[Job- 8581] Data Engineer Pleno,2022-10-05 01:41:53,2022-11-19 00:00:00,"CI&amp;amp;T é uma digital specialist, parceir...","['Airflow', 'Dataproc', 'ELT', 'ETL', 'GCP', '..."
3751,NaN,NaN,NaN,NaN,Research Scientist in ML for Climate Modeling,2022-10-05 01:20:29,2022-11-19 00:00:00,Remote hybrid: Persons in these roles are expe...,"['Machine Learning', 'Mathematics', 'NLP', 'Py..."
3752,NaN,NaN,NaN,NaN,Software Engineer - Market Data Team,2022-10-05 00:14:03,2022-11-19 00:00:00,We are seeking a Market Data Engineer to help ...,"['Big Data', 'Git', 'OOP', 'Research', 'SQL', ..."


In [125]:
concat.rename(columns={0: 'minSalary', 1: 'maxSalary', 2: 'currency', 3: 'salaryPeriod', 4: 'position', 5: 'datePosted', 6: 'datePosted', 7: 'jobDescription', 8: 'skills'}, inplace=True)

In [127]:
concat.to_csv('descriptions_all.csv')

### 6. Merging everything

In [128]:
aijobs = pd.read_csv('aijobs_all.csv')

In [129]:
aijobs

,Unnamed: 0,url,title,location,type,level,salaryRange,salary,company
0,0,https://ai-jobs.net/job/32218-data-science-con...,View full details of `Data Science Content Int...,Remote,Internship,Entry-level,USD 9K - 11K,NaN,NannyML
1,1,https://ai-jobs.net/job/32029-python-php-senio...,View full details of `Python / PHP Senior Deve...,"Remote, EU",Full Time,Senior-level,EUR 65K - 80K,NaN,Beatopia
2,2,https://ai-jobs.net/job/30723-data-scientist-r...,View full details of `Data Scientist - Researc...,"Remote, Hybrid Available (Chicago)",Full Time,Senior-level,USD 55K - 120K,NaN,"HFR, Inc."
3,3,https://ai-jobs.net/job/30166-postdoctoral-res...,View full details of `Postdoctoral Research As...,"Oak Ridge, TN, United States",Temporary,Mid-level,NaN,NaN,Oak Ridge National Laboratory
4,4,https://ai-jobs.net/job/30091-data-science-and...,View full details of `Data Science and Machine...,Remote- USA,Full Time,Senior-level,USD 150K - 200K,NaN,Libertas Funding
...,...,...,...,...,...,...,...,...,...
3749,1749,https://ai-jobs.net/job/29203-fleet-data-manag...,View full details of `Fleet Data Manager SME` ...,"San Diego, California, United States",Full Time,Mid-level,NaN,NaN,Sigma Defense
3750,1750,https://ai-jobs.net/job/29185-job-8581-data-en...,View full details of `[Job- 8581] Data Enginee...,Brazil,Full Time,Senior-level,USD 76K - 150K *,NaN,CI&T
3751,1751,https://ai-jobs.net/job/29144-research-scienti...,View full details of `Research Scientist in ML...,Remote Hybrid,Full Time,Senior-level,USD 56K - 96K *,NaN,The Allen Institute for AI
3752,1752,https://ai-jobs.net/job/29133-software-enginee...,View full details of `Software Engineer - Mark...,"London, England, United Kingdom",Full Time,Mid-level,NaN,NaN,TradingHub


In [130]:
dataset = pd.concat([aijobs, concat], axis = 1)

In [131]:
dataset

,Unnamed: 0,url,title,location,type,level,salaryRange,salary,company,minSalary,maxSalary,currency,salaryPeriod,position,datePosted,datePosted,jobDescription,skills
0,0,https://ai-jobs.net/job/32218-data-science-con...,View full details of `Data Science Content Int...,Remote,Internship,Entry-level,USD 9K - 11K,NaN,NannyML,9000.0,11000.0,USD,YEAR,Data Science Content Intern (Remote),2022-11-09 22:55:46,2022-12-24 00:00:00,Description NannyML - creators of an Open Sour...,"['Bayesian', 'Content creation', 'GitHub', 'Ma..."
1,1,https://ai-jobs.net/job/32029-python-php-senio...,View full details of `Python / PHP Senior Deve...,"Remote, EU",Full Time,Senior-level,EUR 65K - 80K,NaN,Beatopia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['APIs', 'GCP', 'Google Cloud', 'Machine Learn..."
2,2,https://ai-jobs.net/job/30723-data-scientist-r...,View full details of `Data Scientist - Researc...,"Remote, Hybrid Available (Chicago)",Full Time,Senior-level,USD 55K - 120K,NaN,"HFR, Inc.",55000.0,120000.0,USD,YEAR,"Data Scientist - Research, Development &amp; C...",2022-10-26 20:46:21,2022-12-10 00:00:00,"Company Description: HFR, Inc. (HFR) was fo...","['Computer Science', 'Data Analytics', 'Data v..."
3,3,https://ai-jobs.net/job/30166-postdoctoral-res...,View full details of `Postdoctoral Research As...,"Oak Ridge, TN, United States",Temporary,Mid-level,NaN,NaN,Oak Ridge National Laboratory,NaN,NaN,NaN,NaN,Postdoctoral Research Associate - Boundary Pla...,2022-10-21 20:53:22,2022-12-05 00:00:00,Overview: Oak Ridge National Laboratory is th...,"['CAD', 'Engineering', 'Matlab', 'Physics', 'P..."
4,4,https://ai-jobs.net/job/30091-data-science-and...,View full details of `Data Science and Machine...,Remote- USA,Full Time,Senior-level,USD 150K - 200K,NaN,Libertas Funding,150000.0,200000.0,USD,YEAR,Data Science and Machine Learning Manager,2022-10-21 02:15:41,2022-12-05 00:00:00,USA Remote Position POSITION SUMMARY: The Dat...,"['Agile', 'Airflow', 'APIs', 'AWS', 'CI/CD', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,1749,https://ai-jobs.net/job/29203-fleet-data-manag...,View full details of `Fleet Data Manager SME` ...,"San Diego, California, United States",Full Time,Mid-level,NaN,NaN,Sigma Defense,NaN,NaN,NaN,NaN,Fleet Data Manager SME,2022-10-05 01:55:47,2022-11-19 00:00:00,"SOLUTE, Inc., a Sigma Defense Systems company,...","['Data management', 'Security']"
3750,1750,https://ai-jobs.net/job/29185-job-8581-data-en...,View full details of `[Job- 8581] Data Enginee...,Brazil,Full Time,Senior-level,USD 76K - 150K *,NaN,CI&T,NaN,NaN,NaN,NaN,[Job- 8581] Data Engineer Pleno,2022-10-05 01:41:53,2022-11-19 00:00:00,"CI&amp;amp;T é uma digital specialist, parceir...","['Airflow', 'Dataproc', 'ELT', 'ETL', 'GCP', '..."
3751,1751,https://ai-jobs.net/job/29144-research-scienti...,View full details of `Research Scientist in ML...,Remote Hybrid,Full Time,Senior-level,USD 56K - 96K *,NaN,The Allen Institute for AI,NaN,NaN,NaN,NaN,Research Scientist in ML for Climate Modeling,2022-10-05 01:20:29,2022-11-19 00:00:00,Remote hybrid: Persons in these roles are expe...,"['Machine Learning', 'Mathematics', 'NLP', 'Py..."
3752,1752,https://ai-jobs.net/job/29133-software-enginee...,View full details of `Software Engineer - Mark...,"London, England, United Kingdom",Full Time,Mid-level,NaN,NaN,TradingHub,NaN,NaN,NaN,NaN,Software Engineer - Market Data Team,2022-10-05 00:14:03,2022-11-19 00:00:00,We are seeking a Market Data Engineer to help ...,"['Big Data', 'Git', 'OOP', 'Research', 'SQL', ..."


In [133]:
dataset.to_csv('full_dataset.csv')

In [137]:
import os
os.getcwd()

'/home/jovyan/APID Rhizome'

In [138]:
dataset.to_csv('/home/jovyan/__RHI/full_dataset.csv')